In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import random
import pickle

In [3]:
from nltk.corpus import twitter_samples
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk import NaiveBayesClassifier
from nltk import classify

In [4]:
def clean_data(token):
    return [item for item in token if not item.startswith("http") and not item.startswith("@")]


def lemmatization(token):
    lemmatizer = WordNetLemmatizer()

    result = []
    for token, tag in pos_tag(token):
        tag = tag[0].lower()
        token = token.lower()
        if tag in "nva":
            result.append(lemmatizer.lemmatize(token, pos=tag))
        else:
            result.append(lemmatizer.lemmatize(token))
    return result


def remove_stop_words(token, stop_words):
    return [item for item in token if item not in stop_words]


def transform(token):
    result = {}
    for item in token:
        result[item] = True
    return result

In [5]:
def main():
    
    positive_tweets_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweets_tokens = twitter_samples.tokenized('negative_tweets.json')


    stop_words = stopwords.words('english')
    positive_tweets_tokens_cleaned = [remove_stop_words(lemmatization(clean_data(token)), stop_words) for token in positive_tweets_tokens]
    negative_tweets_tokens_cleaned = [remove_stop_words(lemmatization(clean_data(token)), stop_words) for token in negative_tweets_tokens]


    positive_tweets_tokens_transformed = [(transform(token), "Positive") for token in positive_tweets_tokens_cleaned]
    negative_tweets_tokens_transformed = [(transform(token), "Negative") for token in negative_tweets_tokens_cleaned]


    
    dataset = positive_tweets_tokens_transformed + negative_tweets_tokens_transformed
    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    
    classifier = NaiveBayesClassifier.train(train_data)

    
    print("Accuracy is:", classify.accuracy(classifier, test_data))
    print(classifier.show_most_informative_features(10))

    
    f = open('my_classifier.pickle', 'wb')
    pickle.dump(classifier, f)
    f.close()


if __name__ == "__main__":
    main()

Accuracy is: 0.9976666666666667
Most Informative Features
                      :( = True           Negati : Positi =   2053.7 : 1.0
                  arrive = True           Positi : Negati =     32.1 : 1.0
                     sad = True           Negati : Positi =     30.2 : 1.0
                follower = True           Positi : Negati =     23.7 : 1.0
                     bam = True           Positi : Negati =     20.6 : 1.0
                     x15 = True           Negati : Positi =     16.8 : 1.0
              appreciate = True           Positi : Negati =     14.5 : 1.0
               community = True           Positi : Negati =     14.5 : 1.0
                    hurt = True           Negati : Positi =     13.2 : 1.0
                    miss = True           Negati : Positi =     12.2 : 1.0
None


In [6]:
import pickle
import tweepy


def get_twitter_api():
    
    consumer_key = "uPRgRGfmgoPHJyuitn84v9hfk"
    consumer_secret = "IyjACTqz6Nn4QgaiBVhaqeTeFSZbbPa405CAb7526ZNddVjpUX"
    access_token = "949625398249144320-VUEtMbeltQ41giNp5er7IrXBvhtvB3t"
    access_token_secret = "rIJrXrhZKZaS6OAE5qLwaLPLRg0yUDyn554N8R8d57yog"

    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

    
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    return api


def tokenize(tweet):
    return clean_data(tweet)



def get_classifier(pickle_name):
    f = open(pickle_name, 'rb')
    classifier = pickle.load(f)
    f.close()
    return classifier


def find_mood(search):
    classifier = get_classifier('my_classifier.pickle')

    api = get_twitter_api()

    stat = {
        "Positive": 0,
        "Negative": 0
    }
    for tweet in tweepy.Cursor(api.search, q=search).items(1000):
        custom_tokens = tokenize(tweet.text)

        category = classifier.classify(dict([token, True] for token in custom_tokens))
        stat[category] += 1

    print("The mood of", search)
    print(" - Positive", stat["Positive"], round(stat["Positive"]*100/(stat["Positive"] + stat["Negative"]), 1))
    print(" - Negative", stat["Negative"], round(stat["Negative"]*100/(stat["Positive"] + stat["Negative"]), 1))


if __name__ == "__main__":

    find_mood("#python")


The mood of #python
 - Positive 351 97.8
 - Negative 8 2.2
